In [1]:
import pandas as pd

In [2]:
import numpy as np

In [247]:
import os

In [4]:
from sklearn.ensemble import GradientBoostingClassifier

In [5]:
from sklearn.cross_validation import KFold

In [6]:
from sklearn.grid_search import GridSearchCV

In [7]:
import time
import datetime

In [38]:
from sklearn.linear_model import LogisticRegression

In [9]:
from sklearn.preprocessing import StandardScaler

In [139]:
from sklearn.metrics import roc_auc_score

###### prepare data

In [10]:
features = pd.read_csv('./_features/features.csv', index_col='match_id')

In [184]:
Xtest = pd.read_csv('./_features_test/features_test.csv', index_col='match_id')

###### drop final attributes

In [11]:
X = features.drop(features.columns[-6:],1)

###### found skips

In [12]:
counts=X.count()

In [13]:
skips=counts[counts.values < 97230].keys()

###### replace na items to ZERO

In [14]:
X_na = X.fillna(0)

###### target var - radiant_win

In [15]:
y=features.radiant_win

###### create crossval generator

In [16]:
cv = KFold(y.size,shuffle=True, n_folds=5)

###### drop categorial and hero features

In [318]:
keys=list(X.keys()[pd.Series(X.keys()).str.contains('hero', na=False)])

keys=keys+['lobby_type'] #,'start_time']+X.keys()[pd.Series(X.keys()).str.contains('first_blood_player', na=False)].tolist()

Xdr = X_na.drop(keys,1)

###### transform scale

In [319]:
scdr=StandardScaler()

Xsdr=scdr.fit_transform(Xdr)

###### dummy heroes features

In [315]:
N=112

In [316]:
X_pick = np.zeros((X.shape[0], N))

for i, match_id in enumerate(X.index):
    for p in xrange(5):
        X_pick[i, X.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

###### concat scale and dummy hero features

In [320]:
XX=np.column_stack((Xsdr,X_pick))

###### create and fit estimator, with cross validation

###### качество на всех исходных признаках

In [374]:
params={'C':np.linspace(1e-4 ,2)}

In [376]:
grid_no_dummy = GridSearchCV(LogisticRegression(),cv=cv,param_grid=params,n_jobs=-1,scoring='roc_auc')

In [377]:
sc=StandardScaler()

Xs=sc.fit_transform(X_na)

In [378]:
grid_no_dummy.fit(Xs,y)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=97230, n_folds=5, shuffle=True, random_state=None),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([  1.00000e-04,   4.09143e-02,   8.17286e-02,   1.22543e-01,
         1.63357e-01,   2.04171e-01,   2.44986e-01,   2.85800e-01,
         3.26614e-01,   3.67429e-01,   4.08243e-01,   4.49057e-01,
         4.89871e-01,   5.30686e-01,   5.71500e-01,   6.12314e-01,
         6.5312...    1.79593e+00,   1.83674e+00,   1.87756e+00,   1.91837e+00,
         1.95919e+00,   2.00000e+00])},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [379]:
grid_no_dummy.best_score_

0.71632892946425031

In [380]:
grid_no_dummy.best_params_

{'C': 0.040914285714285716}

###### качество после удаления категориальных признаков

In [381]:
grid_no_dummy_dr = GridSearchCV(LogisticRegression(),cv=cv,param_grid=params,n_jobs=-1,scoring='roc_auc')

In [382]:
grid_no_dummy_dr.fit(Xsdr,y)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=97230, n_folds=5, shuffle=True, random_state=None),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([  1.00000e-04,   4.09143e-02,   8.17286e-02,   1.22543e-01,
         1.63357e-01,   2.04171e-01,   2.44986e-01,   2.85800e-01,
         3.26614e-01,   3.67429e-01,   4.08243e-01,   4.49057e-01,
         4.89871e-01,   5.30686e-01,   5.71500e-01,   6.12314e-01,
         6.5312...    1.79593e+00,   1.83674e+00,   1.87756e+00,   1.91837e+00,
         1.95919e+00,   2.00000e+00])},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [383]:
grid_no_dummy_dr.best_score_

0.71642814592328452

In [384]:
grid_no_dummy_dr.best_params_

{'C': 0.040914285714285716}

###### Качество с мешком слов

In [385]:
grid = GridSearchCV(LogisticRegression(),cv=cv,param_grid=params,n_jobs=-1,scoring='roc_auc')

In [386]:
grid.fit(XX,y)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=97230, n_folds=5, shuffle=True, random_state=None),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([  1.00000e-04,   4.09143e-02,   8.17286e-02,   1.22543e-01,
         1.63357e-01,   2.04171e-01,   2.44986e-01,   2.85800e-01,
         3.26614e-01,   3.67429e-01,   4.08243e-01,   4.49057e-01,
         4.89871e-01,   5.30686e-01,   5.71500e-01,   6.12314e-01,
         6.5312...    1.79593e+00,   1.83674e+00,   1.87756e+00,   1.91837e+00,
         1.95919e+00,   2.00000e+00])},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [387]:
grid.best_score_

0.75172924222139459

In [388]:
grid.best_params_

{'C': 0.040914285714285716}

###### predict by test data

In [389]:
estL=LogisticRegression(C=0.040914285714285716)

In [390]:
estL.fit(XX,y)

LogisticRegression(C=0.0409142857143, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

###### transform test data

In [392]:
Xtest_na = Xtest.fillna(0)

In [393]:
Xtest_dr=Xtest_na.drop(keys,1)

In [394]:
tscdr=StandardScaler()

Xtsdr=tscdr.fit_transform(Xtest_dr)

In [192]:
Xt_pick = np.zeros((Xtest.shape[0], N))

for i, match_id in enumerate(Xtest.index):
    for p in xrange(5):
        Xt_pick[i, Xtest.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        Xt_pick[i, Xtest.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [395]:
XXt=np.column_stack((Xtsdr,Xt_pick))

###### predict 

In [396]:
pred=estL.predict_proba(XXt)[:, 1]

In [397]:
pred.max()

0.99637746303166252

In [398]:
pred.min()

0.0085410679923746558

In [399]:
pd.Series(pred).value_counts().max()

1

In [400]:
pred.mean()

0.51819197738485101

In [401]:
pred.tolist()

[0.8229156017587833,
 0.7550621835412348,
 0.1889317957051298,
 0.8642959191285051,
 0.24018528675747863,
 0.3795838663555532,
 0.5274060793445017,
 0.5659853037380305,
 0.21704165202487313,
 0.6678342002577964,
 0.15820205889182956,
 0.32734598645829693,
 0.23523771660288315,
 0.6742491973214475,
 0.5541807451425075,
 0.5850898282970182,
 0.0921248400450938,
 0.5528600053311981,
 0.3195173701860822,
 0.5023079701891732,
 0.7818626017745605,
 0.9289043123893623,
 0.7534939488537822,
 0.9581101157963052,
 0.8811144511244207,
 0.5377647571972386,
 0.7788752705958123,
 0.17818489444677785,
 0.05135369900295805,
 0.789424501252515,
 0.2547892327137811,
 0.8388366540794214,
 0.738328584266813,
 0.5556581495397116,
 0.5767368642157685,
 0.5993193038190471,
 0.5576699893723351,
 0.361807473220683,
 0.6798728090014716,
 0.147471748596843,
 0.117017807040779,
 0.5468872560668654,
 0.5529208105802311,
 0.07654221136750938,
 0.41869959038391974,
 0.7159481463774487,
 0.16886954249645061,
 0.49993